In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow import keras

train_data = np.load('train_data.npz')
valid_data = np.load('valid_data.npz')
test_data = np.load('test_data.npz')

In [2]:
X_train, y_train = train_data['x'], train_data['y']
X_valid, y_valid = valid_data['x'], valid_data['y']
X_test, y_test = test_data['x'], test_data['y']


X_mfcc_train, X_mel_train = X_train[:, :, :20], X_train[:, :, 20:]
X_mfcc_valid, X_mel_valid = X_valid[:, :, :20], X_valid[:, :, 20:]
X_mfcc_test, X_mel_test = X_test[:, :, :20], X_test[:, :, 20:]

In [3]:
from functools import partial
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, 
                                     Flatten, Dense, Dropout, RepeatVector,
                                     LayerNormalization, BatchNormalization)

# drop 0th coefficient as it only conveys a constant offset
X_mfcc_train_f = np.delete(X_mfcc_train, 0, axis=2)
X_mfcc_valid_f = np.delete(X_mfcc_valid, 0, axis=2)
X_mfcc_test_f = np.delete(X_mfcc_test, 0, axis=2)
conv_layer = partial(Conv1D, kernel_size=4, activation='relu', padding='SAME', strides=1)
pooling = partial(MaxPooling1D, pool_size=4, strides=4, padding='SAME')

model = keras.models.Sequential([
    LayerNormalization(),
    conv_layer(input_shape=(1920, 19), filters=32, kernel_size=6),
    pooling(),
    conv_layer(filters=64),
    pooling(),
    conv_layer(filters=128),
    pooling(),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dropout(0.4),
    Dense(units=2)
])

In [4]:
model_path = 'best_model2/'

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True, filepath=model_path)

optimizer = keras.optimizers.Adam(lr=0.00001)
model.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])
history = model.fit(X_mfcc_train_f, y_train, 
                    epochs=30,
                    validation_data=(X_mfcc_valid_f, y_valid), 
                    batch_size=16, 
                    callbacks=[early_stopping, checkpoint])  

Epoch 1/30
61/61 [==============================] - ETA: 0s - loss: 3.8543 - mean_squared_error: 17.6026WARNING:tensorflow:From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: best_model2/assets
61/61 [==============================] - 2s 31ms/step - loss: 3.8543 - mean_squared_error: 17.6026 - val_loss: 2.5461 - val_mean_squared_error: 8.5005
Epoch 2/30
61/61 [==============================] - 1s 22ms/step - loss: 1.7503 - mean_squared_error: 4.6597 - val_loss: 0.9703 - val_mean_squared_error: 1.3870
Epoch 3/30
61/61 [==============================] - 1s 22ms/step - loss: 1.1642 - mean_squared_error: 2.0925 - val_loss: 0.9398 - val_mean_squared_error: 1.3209
Epoch 4/30
6

In [5]:
from sklearn.metrics import accuracy_score, classification_report

def categorize_val_arousal(data):
    cat_data = []
    # A: active, P: passive
    # P: positive, N: negative
    for point in data:
        valence, arousal = point[0], point[1]
        if valence >= 5 and arousal >= 5:
            cat_data.append('AP')
        elif valence < 5 and arousal >= 5:
            cat_data.append('AN')
        elif valence >= 5 and arousal < 5:
            cat_data.append('PP')
        elif valence < 5 and arousal < 5:
            cat_data.append('PN')
    return np.array(cat_data, dtype='str')

score = model.evaluate(X_mfcc_test_f, y_test)
print(f'Test reg RMSE: {list(map(math.sqrt, score))}')
y_pred = model.predict(X_mfcc_test_f)

cat_y_pred = categorize_val_arousal(y_pred)
cat_y_test = categorize_val_arousal(y_test)

accuracy = accuracy_score(cat_y_test, cat_y_pred)
print(f'Test Accuracy: {accuracy}')

12/12 [==============================] - 0s 6ms/step - loss: 0.8593 - mean_squared_error: 1.1218
Test reg RMSE: [0.9269806379952489, 1.059131218421822]
Test Accuracy: 0.5484764542936288


In [6]:
mel_model = keras.models.Sequential([
    LayerNormalization(),
    conv_layer(input_shape=(1920, 128), filters=32, kernel_size=6),
    pooling(),
    conv_layer(filters=64),
    pooling(),
    conv_layer(filters=128),
    pooling(),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dropout(0.4),
    Dense(units=2)
])

checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True, filepath=model_path)

optimizer = keras.optimizers.Adam(lr=0.00001)
mel_model.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])
history = mel_model.fit(X_mel_train, y_train, 
                    epochs=30,
                    validation_data=(X_mel_valid, y_valid), 
                    batch_size=16, 
                    callbacks=[early_stopping, checkpoint])  

Epoch 1/30
61/61 [==============================] - 3s 50ms/step - loss: 3.1322 - mean_squared_error: 13.2128 - val_loss: 1.0253 - val_mean_squared_error: 1.5791
Epoch 2/30
61/61 [==============================] - 3s 51ms/step - loss: 1.2801 - mean_squared_error: 2.5737 - val_loss: 0.8976 - val_mean_squared_error: 1.2447
Epoch 3/30
61/61 [==============================] - 3s 48ms/step - loss: 1.2265 - mean_squared_error: 2.3675 - val_loss: 0.8870 - val_mean_squared_error: 1.2115
Epoch 4/30
61/61 [==============================] - 3s 47ms/step - loss: 1.1421 - mean_squared_error: 2.1001 - val_loss: 0.8691 - val_mean_squared_error: 1.1672
Epoch 5/30
61/61 [==============================] - 3s 50ms/step - loss: 1.1627 - mean_squared_error: 2.0890 - val_loss: 0.8621 - val_mean_squared_error: 1.1480
Epoch 6/30
61/61 [==============================] - 2s 32ms/step - loss: 1.1389 - mean_squared_error: 2.0470 - val_loss: 0.8631 - val_mean_squared_error: 1.1479
Epoch 7/30
61/61 [===============

In [7]:
score = mel_model.evaluate(X_mel_test, y_test)
print(f'Test reg RMSE: {list(map(math.sqrt, score))}')
y_pred = mel_model.predict(X_mel_test)

cat_y_pred = categorize_val_arousal(y_pred)
cat_y_test = categorize_val_arousal(y_test)

accuracy = accuracy_score(cat_y_test, cat_y_pred)
print(f'Test Accuracy: {accuracy}')

12/12 [==============================] - 0s 13ms/step - loss: 0.8654 - mean_squared_error: 1.1593
Test reg RMSE: [0.9302930702894494, 1.0766989633914623]
Test Accuracy: 0.5096952908587258


In [8]:
from sklearn.preprocessing import LabelEncoder

cat_model = keras.models.Sequential([
    LayerNormalization(),
    conv_layer(input_shape=(1920, 19), filters=64, kernel_size=4),
    pooling(),
    conv_layer(filters=128),
    pooling(),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=4, activation='softmax')
])

cat_y_train, cat_y_valid, cat_y_test = categorize_val_arousal(y_train), categorize_val_arousal(y_valid), categorize_val_arousal(y_test)
encoder = LabelEncoder()
encoder.fit(cat_y_train)
encoded_y_train = encoder.transform(cat_y_train)
encoded_y_valid = encoder.transform(cat_y_valid)
encoded_y_test = encoder.transform(cat_y_test)

optimzer = keras.optimizers.Adam(lr=0.00001)
cat_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = cat_model.fit(X_mfcc_train_f, encoded_y_train, 
                    epochs=30,
                    validation_data=(X_mfcc_valid_f, encoded_y_valid), 
                    batch_size=32)

Epoch 1/30
31/31 [==============================] - 0s 15ms/step - loss: 1.6556 - accuracy: 0.4352 - val_loss: 1.1798 - val_accuracy: 0.5126
Epoch 2/30
31/31 [==============================] - 0s 10ms/step - loss: 1.1303 - accuracy: 0.5606 - val_loss: 1.1974 - val_accuracy: 0.5189
Epoch 3/30
31/31 [==============================] - 0s 10ms/step - loss: 1.0526 - accuracy: 0.6083 - val_loss: 1.1398 - val_accuracy: 0.5735
Epoch 4/30
31/31 [==============================] - 0s 10ms/step - loss: 1.0102 - accuracy: 0.6207 - val_loss: 1.1518 - val_accuracy: 0.5441
Epoch 5/30
31/31 [==============================] - 0s 10ms/step - loss: 0.9209 - accuracy: 0.6622 - val_loss: 1.1430 - val_accuracy: 0.5504
Epoch 6/30
31/31 [==============================] - 0s 10ms/step - loss: 0.8743 - accuracy: 0.6611 - val_loss: 1.2247 - val_accuracy: 0.5357
Epoch 7/30
31/31 [==============================] - 0s 10ms/step - loss: 0.7781 - accuracy: 0.7078 - val_loss: 1.2672 - val_accuracy: 0.5504
Epoch 8/30
31